In [2]:
import numpy as np
import pandas as pd

import xgboost

import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

from sklearn.feature_selection import SelectKBest
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

df = pd.read_csv('train.csv')
df.drop('Id',axis=1, inplace=True)
df_test = pd.read_csv('test.csv')
df_test.drop('Id',axis=1, inplace=True)

In [ ]:
df

In [4]:
df_copy = df.copy()

In [5]:
df_numeric = df.select_dtypes(exclude=['object'])
df_test_numeric = df_test.select_dtypes(exclude=['object'])
df_cat = df.select_dtypes(include=['object'])

In [18]:
df_cat

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,No,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,No,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,No,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,No,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,No,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,No,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1456,RL,Pave,No,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1457,RL,Pave,No,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal
1458,RL,Pave,No,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [20]:
df_cat['Alley'].fillna('No', inplace=True)
df_cat['BsmtQual'].fillna('No', inplace=True)
df_cat['BsmtCond'].fillna('No', inplace=True)
df_cat['BsmtExposure'].fillna('No', inplace=True)
df_cat['MasVnrType'].fillna('No', inplace=True)
df_cat['Electrical'].fillna('SBrkr', inplace=True)
df_cat['FireplaceQu'].fillna('No', inplace=True)
df_cat['GarageType'].fillna('No', inplace=True)
df_cat['GarageFinish'].fillna('No', inplace=True)
df_cat['GarageQual'].fillna('No', inplace=True)
df_cat['GarageCond'].fillna('No', inplace=True)
df_cat['PoolQC'].fillna('No', inplace=True)
df_cat['Fence'].fillna('No', inplace=True)

C:\Users\NetDriver\AppData\Local\Temp\ipykernel_9140\1859693478.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cat['Alley'].fillna('No', inplace=True)
C:\Users\NetDriver\AppData\Local\Temp\ipykernel_9140\1859693478.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [36]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df_cat['BsmtFinType1'])
le.classes_
BsmtFinType1 = le.transform(df_cat['BsmtFinType1'])
BsmtFinType1_df = pd.DataFrame(BsmtFinType1, columns=['BsmtFinType1_enum'])

le.fit(df_cat['BsmtFinType2'])
le.classes_
BsmtFinType2 = le.transform(df_cat['BsmtFinType2'])
BsmtFinType2_df = pd.DataFrame(BsmtFinType2, columns=['BsmtFinType2_enum'])

MiscFeature = pd.get_dummies(df_cat['MiscFeature'], columns=['MiscFeature'])

df_combined = pd.concat([BsmtFinType1_df, BsmtFinType2_df, MiscFeature], axis=1)

In [ ]:
df_combined

In [34]:
MiscFeature

,Gar2,Othr,Shed,TenC
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
1455,False,False,False,False
1456,False,False,False,False
1457,False,False,True,False
1458,False,False,False,False


In [37]:
df = pd.concat([df_combined,df_cat], axis=1)

# Функция для обработки категориальных признаков
def encode_categorical_features(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            unique_values = df[column].nunique()
            if unique_values <= 5:
                # Применяем one-hot encoding
                df = pd.get_dummies(df, columns=[column])
            else:
                # Применяем label encoding
                le = LabelEncoder()
                df[column] = le.fit_transform(df[column])
    return df

# Применяем функцию к нашему DataFrame
df_encoded = encode_categorical_features(df)

print(df_encoded)

      BsmtFinType1_enum  BsmtFinType2_enum   Gar2   Othr   Shed   TenC  \
0                     2                  5  False  False  False  False   
1                     0                  5  False  False  False  False   
2                     2                  5  False  False  False  False   
3                     0                  5  False  False  False  False   
4                     2                  5  False  False  False  False   
...                 ...                ...    ...    ...    ...    ...   
1455                  5                  5  False  False  False  False   
1456                  0                  4  False  False  False  False   
1457                  2                  5  False  False   True  False   
1458                  2                  4  False  False  False  False   
1459                  1                  3  False  False  False  False   

      Neighborhood  Condition1  Condition2  HouseStyle  ...  PoolQC_No  \
0                5           2       


BsmtFinType1 - 1234567
BsmtFinType2 - 1234567
MiscFeature - OneHot


In [39]:
estimator = RandomForestRegressor(n_estimators=10, random_state=0)
imputer = IterativeImputer(estimator=estimator, max_iter=10, random_state=0)
imputed_df = df_encoded
imputed_df

,BsmtFinType1_enum,BsmtFinType2_enum,Gar2,Othr,Shed,TenC,Neighborhood,Condition1,Condition2,HouseStyle,...,PoolQC_No,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_No,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC
0,2,5,False,False,False,False,5,2,2,5,...,True,False,False,False,False,True,False,False,False,False
1,0,5,False,False,False,False,24,1,2,2,...,True,False,False,False,False,True,False,False,False,False
2,2,5,False,False,False,False,5,2,2,5,...,True,False,False,False,False,True,False,False,False,False
3,0,5,False,False,False,False,6,2,2,5,...,True,False,False,False,False,True,False,False,False,False
4,2,5,False,False,False,False,15,2,2,5,...,True,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,5,5,False,False,False,False,8,2,2,5,...,True,False,False,False,False,True,False,False,False,False
1456,0,4,False,False,False,False,14,2,2,2,...,True,False,False,True,False,False,False,False,False,False
1457,2,5,False,False,True,False,6,2,2,5,...,True,True,False,False,False,False,False,False,True,False
1458,2,4,False,False,False,False,12,2,2,2,...,True,False,False,False,False,True,False,False,False,False


In [40]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

data = imputed_df.copy()

contamination = 0.03  # предполагаемая доля выбросов
model = IsolationForest(contamination=contamination, random_state=42)
model.fit(data)

predictions = model.predict(data)

# Замена выбросов на NaN
df_no_outliers = data.copy()
# Преобразуем 1D предсказания в маску для замены выбросов
mask = predictions == -1
for column in df_no_outliers.columns:
    df_no_outliers.loc[mask, column] = np.nan

df_no_outliers

C:\Users\NetDriver\AppData\Local\Temp\ipykernel_9140\578992405.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_no_outliers.loc[mask, column] = np.nan
C:\Users\NetDriver\AppData\Local\Temp\ipykernel_9140\578992405.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_no_outliers.loc[mask, column] = np.nan
C:\Users\NetDriver\AppData\Local\Temp\ipykernel_9140\578992405.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_no_outliers.loc[mask, column] = np.na

,BsmtFinType1_enum,BsmtFinType2_enum,Gar2,Othr,Shed,TenC,Neighborhood,Condition1,Condition2,HouseStyle,...,PoolQC_No,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_No,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC
0,2.0,5.0,False,False,False,False,5.0,2.0,2.0,5.0,...,True,False,False,False,False,True,False,False,False,False
1,0.0,5.0,False,False,False,False,24.0,1.0,2.0,2.0,...,True,False,False,False,False,True,False,False,False,False
2,2.0,5.0,False,False,False,False,5.0,2.0,2.0,5.0,...,True,False,False,False,False,True,False,False,False,False
3,0.0,5.0,False,False,False,False,6.0,2.0,2.0,5.0,...,True,False,False,False,False,True,False,False,False,False
4,2.0,5.0,False,False,False,False,15.0,2.0,2.0,5.0,...,True,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,5.0,5.0,False,False,False,False,8.0,2.0,2.0,5.0,...,True,False,False,False,False,True,False,False,False,False
1456,0.0,4.0,False,False,False,False,14.0,2.0,2.0,2.0,...,True,False,False,True,False,False,False,False,False,False
1457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1458,2.0,4.0,False,False,False,False,12.0,2.0,2.0,2.0,...,True,False,False,False,False,True,False,False,False,False


In [41]:
df_no_outliers

,BsmtFinType1_enum,BsmtFinType2_enum,Gar2,Othr,Shed,TenC,Neighborhood,Condition1,Condition2,HouseStyle,...,PoolQC_No,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_No,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC
0,2.0,5.0,False,False,False,False,5.0,2.0,2.0,5.0,...,True,False,False,False,False,True,False,False,False,False
1,0.0,5.0,False,False,False,False,24.0,1.0,2.0,2.0,...,True,False,False,False,False,True,False,False,False,False
2,2.0,5.0,False,False,False,False,5.0,2.0,2.0,5.0,...,True,False,False,False,False,True,False,False,False,False
3,0.0,5.0,False,False,False,False,6.0,2.0,2.0,5.0,...,True,False,False,False,False,True,False,False,False,False
4,2.0,5.0,False,False,False,False,15.0,2.0,2.0,5.0,...,True,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,5.0,5.0,False,False,False,False,8.0,2.0,2.0,5.0,...,True,False,False,False,False,True,False,False,False,False
1456,0.0,4.0,False,False,False,False,14.0,2.0,2.0,2.0,...,True,False,False,True,False,False,False,False,False,False
1457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1458,2.0,4.0,False,False,False,False,12.0,2.0,2.0,2.0,...,True,False,False,False,False,True,False,False,False,False


In [42]:
print("Оригинальный DataFrame:")
print(df)

df_no_outliers = replace_outliers_with_nan(df)

print("\nDataFrame после замены выбросов на NaN:")
df_no_outliers

Оригинальный DataFrame:
      BsmtFinType1_enum  BsmtFinType2_enum   Gar2   Othr   Shed   TenC  \
0                     2                  5  False  False  False  False   
1                     0                  5  False  False  False  False   
2                     2                  5  False  False  False  False   
3                     0                  5  False  False  False  False   
4                     2                  5  False  False  False  False   
...                 ...                ...    ...    ...    ...    ...   
1455                  5                  5  False  False  False  False   
1456                  0                  4  False  False  False  False   
1457                  2                  5  False  False   True  False   
1458                  2                  4  False  False  False  False   
1459                  1                  3  False  False  False  False   

     MSZoning Street Alley LotShape  ... GarageType GarageFinish GarageQual  \
0       

NameError: name 'replace_outliers_with_nan' is not defined